# Connectome Analyses

We develop tools to analyze a circuit model's connectome.

In [1]:
import os
import numpy as np
import pandas as pd
from neuro_dmt.library.users.visood.sscx_dissemination.analyses.composition\
    import CompositionAnalysesSuite
from neuro_dmt.library.users.visood.sscx_dissemination.analyses.connectome\
    import ConnectomeAnalysesSuite
from neuro_dmt.models.bluebrain.circuit.atlas import\
    BlueBrainCircuitAtlas
from neuro_dmt.models.bluebrain.circuit.model import\
    BlueBrainCircuitModel,\
    CircuitProvenance
from neuro_dmt.models.bluebrain.circuit.adapter import\
    BlueBrainCircuitAdapter

('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'roi', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'roi', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'roi', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mtype', 'etype', 'synapse_class')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'roi', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'roi')


In [2]:
project =\
    os.path.join(
        "/gpfs/bbp.cscs.ch/project")
proj_sscx_diss=\
    os.path.join(
        project,
        "proj83")
data_sscx_diss=\
    os.path.join(
        proj_sscx_diss,
        "data")
atlases_sscx_diss=\
    os.path.join(
        data_sscx_diss,
        "atlas/S1/MEAN")
circuits_sscx_diss=\
    os.path.join(
        proj_sscx_diss,
        "circuits")
atlas_bio_m =\
    BlueBrainCircuitAtlas(
        path=os.path.join(
                atlases_sscx_diss, "P14-MEAN"))
path_bio_m =\
    os.path.join(
        circuits_sscx_diss,
        "Bio_M/20191206")
circuit_bio_m =\
    BlueBrainCircuitModel(
        path_circuit_data=path_bio_m,
            provenance=CircuitProvenance(
                label="SSCxRatDisseminationBioM",
                authors=["BBP Team"],
                release_date="20191212",
                uri=path_bio_m,
                animal="Wistar Rat",
                age="P14",
                brain_region="SSCx"))
adapter =\
    BlueBrainCircuitAdapter()
suite_connectome_analyses =\
    ConnectomeAnalysesSuite(
        sample_size=20)

## Mock circuit

We will use a mock-circuit to test-develop connectome analyses.

In [3]:
from neuro_dmt.models.bluebrain.circuit.mock.circuit import\
    MockCircuit
from neuro_dmt.models.bluebrain.circuit.mock.test.mock_circuit_light import\
    circuit_composition,\
    circuit_connectivity

bluepy_mock_circuit =\
    MockCircuit.build(
        circuit_composition,
        circuit_connectivity)
mock_circuit_model =\
    BlueBrainCircuitModel(
        bluepy_mock_circuit,
        label="BlueBrainCircuitModelMockLight")

# Adapter Functionality

In this section we develop our adapter's functionality

In [4]:
mock_circuit_model

In [5]:
adapter = BlueBrainCircuitAdapter()
cell_types_m = adapter.get_cell_types(mock_circuit_model, ("mtype",))
print(cell_types_m.head())
cell_types_me = adapter.get_cell_types(mock_circuit_model, ("mtype", "etype",) )
print(cell_types_me.head())

     mtype
0   L1_DAC
1  L1_SLAC
2   L23_MC
3  L23_ChC
4  L23_SBC
     mtype etype
0   L1_DAC  bNAC
1   L1_DAC   NAC
2  L1_SLAC  bNAC
3  L1_SLAC   NAC
4   L23_MC  bNAC


In [16]:
pathways = adapter.get_pathways(mock_circuit_model, ("mtype", "etype"))
print(pathways.shape)
assert pathways.shape[0] == cell_types_me.shape[0] ** 2
pathways.head()

(1444, 4)


pre_synaptic_cell       post_synaptic_cell      
              mtype etype              mtype etype
0            L1_DAC  bNAC             L1_DAC  bNAC
1            L1_DAC  bNAC             L1_DAC   NAC
2            L1_DAC  bNAC            L1_SLAC  bNAC
3            L1_DAC  bNAC            L1_SLAC   NAC
4            L1_DAC  bNAC             L23_MC  bNAC

In [20]:
adapter.get_cells(mock_circuit_model,
                  **pathways.iloc[1].post_synaptic_cell)\
        .etype\
        .unique()

array(['NAC'], dtype=object)

# Number of Afferent Connections

We develop an analysis of the number of afferent connections

In [ ]:
from neuro_dmt.analysis.reporting import CheetahReporter
reporter_analyses =\
    CheetahReporter(
        path_output_folder=os.path.join(
            suite_connectome_analyses.path_reports,
            "analyses"))

We can also run our analyses over a chosen set of cell-types.

In [5]:
post_synaptic_cell =\
    suite_connectome_analyses\
        .get_random_cells(
            mock_circuit_model,
            adapter,
            {"mtype": "L23_MC"})\
        .iloc[0]
post_synaptic_cell

region                  S1FL
layer                      2
nucleus          not-defined
mtype                 L23_MC
etype            not-defined
morph_class      not-defined
synapse_class            INH
x                    110.097
y                    168.212
z                    77.1217
gid                    10446
Name: 10446, dtype: object

In [6]:
suite_connectome_analyses =\
    ConnectomeAnalysesSuite(
        sample_size=20,
        cell_mtypes=["L5_TPC:A", "L5_TPC:B", "L5_UPC"])
m = suite_connectome_analyses\
    .number_afferent_connections(
        mock_circuit_model, adapter,
        {"mtype": "L23_MC"})

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
INFO@<2020-01-28 16:41:37>                                                      

            Number afferent connections for post-synaptic cell type: {'mtype': 'L23_MC'}
            

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
INFO@<2020-01-28 16:41:37>                                                      
number_afferent_connections call count : 1



In [7]:
m.head()

(pre_synaptic_cell_type, mtype)  soma_distance
L1_DAC                           350.0            1.0
                                 450.0            1.0
L1_SLAC                          550.0            1.0
L23_ChC                          250.0            1.0
L23_MC                           150.0            2.0
Name: number_connections_afferent, dtype: float64

## SSCx variation

In [ ]:
report_connectome_analyses_bio_m =\
    suite_connectome_analyses\
        .analysis_number_afferent_connections(
            circuit_bio_m,
            adapter)


Because 60 post-synaptic mtypes can take too long, we can set the number of 
mtypes to process in one run:

In [ ]:
def get_layer_mtypes(circuit, query_layer):
    mtypes_all =\
        pd.Series(adapter.get_mtypes(circuit), name="mtype")
    return\
        list(mtypes_all[
                [query_layer in mtype for mtype in mtypes_all]
        ].values)
def analyze_number_afferent_connections(
        circuit,
        adapter,
        selected_mtypes,
        reporter,
        **kwargs):
    """..."""
    suite_connectome_analyses =\
        ConnectomeAnalysesSuite(
            sample_size=20,
            cell_mtypes=selected_mtypes)
    report=\
        suite_connectome_analyses\
            .analysis_number_afferent_connections(
                circuit, adapter, **kwargs)
    return reporter.post(report)

In [ ]:
analyze_number_afferent_connections(
    mock_circuit_model,
    adapter,
    ["L5_TPC:A", "L5_TPC:B",  "L5_UPC"],
    reporter_analyses,
    sample_size=200)

In [ ]:
suite_connectome_analyses =\
    ConnectomeAnalysesSuite(
        sample_size=20,
        cell_mtypes=["L23_MC"])
measurement_example =\
    suite_connectome_analyses\
        .analysis_number_afferent_connections\
        .get_measurement(
            mock_circuit_model,
            adapter)

In [ ]:
measurement_example["data"]